In [1]:
import logging
from pathlib import Path
from Pegasus.api import *

logging.basicConfig(level = logging.DEBUG)

fa = File("input.txt")
rc = ReplicaCatalog()
rc.add_replica("local", fa, Path(".").resolve() / "input.txt").write()

tc = TransformationCatalog()

thumbs_up = Container(
                "thumbs_up",
                Container.DOCKER,
                image="docker://vedularaghu/thumbs_up:latest"
            )

tc.add_containers(thumbs_up).write()

add_thumbs = Transformation(
                        "add_thumbs",
                        site="condorpool",
                        pfn="/usr/bin/add_thumbs_up.py",
                        is_stageable=False,
                        container=thumbs_up
                )

tc.add_transformations(add_thumbs).write()

wf = Workflow("add_thumbs")

fb = File("output.txt")

job_add_thumbs = Job(add_thumbs)\
                    .add_args(fa, fb)\
                    .add_inputs(fa)\
                    .add_outputs(fb)

wf.add_jobs(job_add_thumbs)
wf.add_replica_catalog(rc)
wf.add_transformation_catalog(tc)

try:
    wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)


Plan:
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.07.14 17:06:43.592 UTC:    
2020.07.14 17:06:43.598 UTC:   ----------------------------------------------------------------------- 
2020.07.14 17:06:43.603 UTC:   File for submitting this DAG to HTCondor           : add_thumbs-0.dag.condor.sub 
2020.07.14 17:06:43.609 UTC:   Log of DAGMan debugging messages                 : add_thumbs-0.dag.dagman.out 
2020.07.14 17:06:43.615 UTC:   Log of HTCondor library output                     :

[##################################################] 100.0% ..Success (Completed: 8, Queued: 0, Running: 0, Failed: 0)


Analyzer:
Your database is compatible with Pegasus version: 5.0.0dev

************************************Summary*************************************

 Submit Directory   : /home/scitech/shared-data/wf-4/scitech/pegasus/add_thumbs/run0018
 Total jobs         :      8 (100.00%)
 # jobs succeeded   :      8 (100.00%)
 # jobs failed      :      0 (0.00%)
 # jobs held        :      0 (0.00%)
 # jobs unsubmitted :      0 (0.00%)

 

Statistics:
Your database is compatible with Pegasus version: 5.0.0dev

#
# Pegasus Workflow Management System - http://pegasus.isi.edu
#
# Workflow summary:
#   Summary of the workflow execution. It shows total
#   tasks/jobs/sub workflows run, how many succeeded/failed etc.
#   In case of hierarchical workflow the calculation shows the
#   statistics across all the sub workflows.It shows the following
#   statistics about tasks, jobs and sub workflows.
#     * Succeeded - total count of succeeded tasks/jobs/sub workflows.
#     * Failed - total count of faile